In [2]:
import pandas as pd

In [4]:
pd.read_csv('data/processed/default_weights.csv').drop('Unnamed: 0', axis = 1)

,variable,weight
0,senior_%,0.421416
1,terrorism_index,0.220592
2,visa_index,0.387819
3,epi_score,0.277032
4,usd_score,0.515279
5,wh_sites,0.820729
6,amusement_parks,0.314179
7,michelin_stars,0.629084
8,cuisine_ranking,0.612919


In [27]:
def find_different_countries(df1, df2):
    df2['country'] = df2['country'].str.capitalize().apply(lambda x: x.title())

    countries_database = set(df1['country'])
    countries_df2 = set(df2['country'])

    common_countries = list(countries_database.intersection(countries_df2))
    print(len(common_countries))

    # Find countries present in df1 but not in df2
    diff_database = list(countries_database.difference(countries_df2))
    # Find countries present in df2 but not in df1
    diff_df2 = list(countries_df2.difference(countries_database))

    print('Database differences:', diff_database)
    print('New variable differences:', diff_df2)

In [28]:
# Tourism
inbounds = pd.read_excel('data/processed/un_inbounds_total.xlsx').drop('Unnamed: 0', axis =1)
purpose = pd.read_excel('data/processed/un_inbounds_purpose_of_trip.xlsx').drop('Unnamed: 0', axis =1)
region = pd.read_excel('data/processed/un_arrivals_by_region.xlsx').drop('Unnamed: 0', axis =1)
acc = pd.read_excel('data/processed/accomodation_variables.xlsx').drop('Unnamed: 0', axis =1)

# Demographics
age = pd.read_excel('data/processed/age_variable.xlsx')

# Activities
amusement = pd.read_csv('data/processed/amusment_parks_variable.csv')
cuisine = pd.read_excel('data/processed/cuisines_variable.xlsx')
michelin = pd.read_excel('data/processed/michelin_variable.xlsx')
wh = pd.read_excel('data/processed/world_heritage_variable.xlsx')

# Environment
env = pd.read_excel('data/processed/environmental_variables.xlsx')

# Safety
gpi = pd.read_excel('data/raw/safety vars/global_peace_index.xlsx').drop('Unnamed: 0', axis = 1)
gti = pd.read_excel('data/raw/safety vars/global_terrorism_index.xlsx').drop('Unnamed: 0', axis = 1)
safety = pd.read_excel('data/processed/safety_variables.xlsx').drop('Unnamed: 0', axis = 1)
visa = pd.read_excel('data/processed/visa_variable.xlsx')

# Historical Data - UN , Age, and Safety-Terrorism

#### joining with un vars

In [29]:
# joining with purpose
purpose = purpose.rename(columns = {'business_tourists': 'inbounds_business', 'personal_tourists': 'inbounds_leisure'})
print(inbounds.shape)
print(inbounds.country.nunique())
print(purpose.shape)
print(purpose.country.nunique())

df = pd.merge(inbounds, purpose, on = ['country', 'year'])
print(df.shape)
print(df.country.nunique())

(6244, 5)
223
(6244, 4)
223
(6244, 7)
223


In [30]:
# joining with arrival region
region = region.rename(columns = {'Africa': 'inbounds_africa', 'Americas': 'inbounds_america', 'East Asia and the Pacific': 'inbounds_east_asia_pacific',
                         'Europe': 'inbounds_europe', 'Middle East': 'inbounds_middle_east', 'Other not classified': 'inbounds_other', 'South Asia': 'inbounds_south_asia'})\
                .drop(['region', 'region_total'], axis = 1)

print(inbounds.shape)
print(inbounds.country.nunique())
print(region.shape)
print(region.country.nunique())

df = pd.merge(inbounds, region, on = ['country', 'year'])
print(df.shape)
print(df.country.nunique())

(6244, 5)
223
(6244, 9)
223
(6244, 12)
223


In [31]:
# joining with accommodation variables
print(inbounds.shape)
print(inbounds.country.nunique())
print(acc.shape)
print(acc.country.nunique())

df = pd.merge(inbounds, acc, on = ['country', 'year'])
print(df.shape)
print(df.country.nunique())

(6244, 5)
223
(6244, 8)
223
(6244, 11)
223


Before joining the database with the rest of variables, we need to standardize country names

In [32]:
df['country'] = df['country'].str.capitalize().apply(lambda x: x.title())

# some adjustments for the final country names:
country_mapping = {
    'Korea, Democratic People´S Republic Of' : 'North Korea',
    'Korea, Republic Of': 'South Korea',
    'Viet Nam': 'Vietnam',
    'Czech Republic (Czechia)': 'Czech Republic',
    'Venezuela, Bolivarian Republic Of': 'Venezuela',
    'Lao People´S Democratic Republic': 'Laos',
    'Cote D´Ivoire': 'Cote DIvoire',
    'Moldova, Republic Of': 'Moldova',
    'Bolivia, Plurinational State Of': 'Bolivia',
    'Tanzania, United Republic Of': 'Tanzania',
    'Iran, Islamic Republic Of': 'Iran',
    'Micronesia, Federated States Of': 'Micronesia',
    'Cote DIvoire': 'Cote Divoire',
    'Serbia And Montenegro': 'Serbia',
    'Russian Federation': 'Russia',
    'Türkiye': 'Turkey'

}

df['country'] = df['country'].replace(country_mapping)

#### Age segmentation

In [33]:
age.country.nunique()

244

In [34]:
age['country'] = age['country'].str.capitalize().apply(lambda x: x.title())

countries_database = set(df['country'])
countries_age = set(age['country'])

common_countries = list(countries_database.intersection(countries_age))
print(len(common_countries))

# Find countries present in df1 but not in df2
diff_database = list(countries_database.difference(countries_age))
# Find countries present in df2 but not in df1
diff_age = list(countries_age.difference(countries_database))

201


In [35]:
# countries present in the database but not in the variable
diff_database

['Reunion',
 'Laos',
 'Tanzania',
 'Sint Eustatius',
 'Turkey',
 'Czech Republic',
 'Congo, Democratic Republic Of The',
 'Moldova',
 'Bonaire',
 'Cote DIvoire',
 'Iran',
 'South Korea',
 'Saba',
 'Vietnam',
 'Bolivia',
 'North Korea',
 'Macao, China',
 'Venezuela',
 'Hong Kong, China',
 'Russia',
 'Taiwan Province Of China']

In [36]:
# renaming countries in age segmentation
country_mapping = {
    
    'Republic Of Korea': 'South Korea',
    'United States of America': 'United States Of America',
    'China, Hong Kong Sar': 'Hong Kong, China',
    'China, Macao Sar': 'Macao, China',
    "Lao People'S Democratic Republic": 'Laos',
    'Venezuela (Bolivarian Republic Of)': 'Venezuela',
    'Czechia': 'Czech Republic',
    'Iran (Islamic Republic Of)': 'Iran',
    'Viet Nam': 'Vietnam',
    "Dem. People'S Republic Of Korea" : 'North Korea',
    'Democratic Republic Of The Congo': 'Congo, Democratic Republic Of The',
    'Bolivia (Plurinational State Of)': 'Bolivia',
    'Republic Of Moldova': 'Moldova',
    'United Republic Of Tanzania': 'Tanzania',
    "Côte D'Ivoire": 'Cote Divoire',
    'China, Taiwan Province Of China': 'Taiwan Province Of China',
    'Réunion': 'Reunion',
    'Bonaire, Sint Eustatius And Saba': 'Bonaire',
    'Russian Federation': 'Russia',
    'Türkiye': 'Turkey'
}

age['country'] = age['country'].replace(country_mapping)

In [37]:
age

,Unnamed: 0,country,year,0-4,5-9,10-14,15-19,20-24,25-29,30-34,...,70-74,75-79,80-84,85-89,90-94,95-99,100+,total,seniors,senior_%
0,1777,Burundi,1995,1093196.5,969105.0,846988.5,658502.5,503237.0,390504.0,308702.5,...,45194.5,26002.5,11444.0,3375.0,843.5,526.0,129.5,5931284.0,151455.5,2553.502749
1,1778,Burundi,1996,1096185.5,979667.0,864261.0,675698.0,494751.5,379458.5,296542.5,...,44503.5,25507.0,11054.0,3057.0,461.0,149.5,87.5,5929808.5,148862.5,2510.409906
2,1779,Burundi,1997,1090480.0,989838.0,876792.5,699136.5,488711.5,369333.0,284483.5,...,43927.0,25140.5,10768.5,2845.0,280.0,0.0,42.0,5923540.0,146969.0,2481.100828
3,1780,Burundi,1998,1082364.5,1004539.0,892859.0,736526.0,512165.0,379393.5,289414.5,...,44123.0,25359.0,10915.0,2967.0,336.0,10.5,20.5,6034973.0,148341.5,2458.030881
4,1781,Burundi,1999,1099350.0,999831.5,908842.5,773521.0,546432.5,393592.0,300156.5,...,44607.0,25760.5,11184.5,3142.5,390.5,13.0,11.5,6179765.0,150762.0,2439.607331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,22362,Tuvalu,2023,1278.0,1172.5,1167.0,986.0,906.5,905.5,900.5,...,215.5,101.5,62.0,25.0,5.0,0.0,0.0,11391.0,756.5,6641.207971
7316,22363,Tuvalu,2024,1276.5,1188.5,1172.5,1020.0,890.5,893.5,903.0,...,220.5,115.5,55.5,28.0,4.0,0.0,0.0,11474.0,801.0,6981.000523
7317,22440,Wallis And Futuna Islands,2022,718.5,881.5,1025.0,1012.0,672.5,580.5,612.0,...,424.5,310.5,186.0,96.5,26.0,6.5,0.0,11540.0,1519.5,13167.244367
7318,22441,Wallis And Futuna Islands,2023,688.5,845.5,1005.0,1019.0,729.0,546.0,585.5,...,432.5,334.5,178.5,107.0,32.5,7.0,0.0,11462.0,1576.5,13754.144128


In [38]:
df_hist = pd.merge(df, age[['year', 'country', 'seniors', 'senior_%']], on=['country', 'year'], how='left')
df_hist

,country,year,inbounds_tourists,inbounds_total,region,avg_length_stay,beds_number,establishments_number,rooms_number,beds_occupancy_rate,rooms_occupancy_rate,seniors,senior_%
0,Afghanistan,1995,NaN,NaN,South Asia,NaN,NaN,NaN,NaN,NaN,NaN,373005.5,2271.985760
1,Afghanistan,1996,NaN,NaN,South Asia,NaN,NaN,NaN,NaN,NaN,NaN,389379.5,2276.379536
2,Afghanistan,1997,NaN,NaN,South Asia,NaN,NaN,NaN,NaN,NaN,NaN,405276.5,2278.452291
3,Afghanistan,1998,NaN,NaN,South Asia,NaN,NaN,NaN,NaN,NaN,NaN,421195.0,2277.764389
4,Afghanistan,1999,NaN,NaN,South Asia,NaN,NaN,NaN,NaN,NaN,NaN,438599.0,2277.108780
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6239,Zimbabwe,2018,NaN,2580000.0,Africa,3.0,12772.0,96.0,6483.0,39.0,53.0,486199.5,3232.137585
6240,Zimbabwe,2019,NaN,2294000.0,Africa,3.0,13158.0,97.0,6676.0,32.0,44.0,503951.5,3284.178333
6241,Zimbabwe,2020,NaN,639000.0,Africa,3.0,14551.0,95.0,7217.0,15.0,19.0,519181.5,3315.377909
6242,Zimbabwe,2021,NaN,381000.0,Africa,3.0,14551.0,95.0,7217.0,18.0,27.0,528314.0,3305.284104


#### Safety Variables

In [39]:
print(df.country.nunique())
print(safety.country.nunique())

222
171


In [40]:
gpi

,code,name,year,rank,peace_index,index_dist,index_poli,index_homi,index_jail,index_wmid,...,index_unfu,index_hewe,index_weex,index_refu,index_rela,index_econ,index_ndec,index_militarisation,index_safety_security,index_domestic_international_conflict
0,AFG,Afghanistan,2008,160,3.104302,3.960,2.500,2.482,1.237,5.0,...,4.200000,1.038494,1.000,5.000,4.0,1.0,1.000,2.593617,3.372441,3.152660
1,AGO,Angola,2008,90,1.990950,3.521,2.500,2.590,1.364,3.0,...,1.539251,1.326102,1.000,1.293,3.0,1.2,1.000,1.744400,2.395206,1.654789
2,ALB,Albania,2008,52,1.790608,2.605,2.701,2.748,2.139,4.0,...,1.544722,1.002420,1.000,1.170,2.0,1.5,1.000,1.776159,2.235735,1.247182
3,ARE,United Arab Emirates,2008,34,1.651118,1.244,3.250,1.390,2.812,2.0,...,1.796144,1.523138,1.202,1.002,1.0,1.0,1.000,2.217629,1.874588,1.000000
4,ARG,Argentina,2008,61,1.820331,3.639,4.436,2.913,2.202,2.0,...,1.472312,1.422613,1.001,1.001,3.0,1.0,1.000,1.398321,2.370653,1.402576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2731,COD,Democratic Republic of the Congo,2023,159,3.213804,3.458,1.500,4.250,1.190,5.0,...,3.441633,1.076000,1.000,3.079,4.0,1.8,1.000,2.063486,3.749412,3.421055
2732,SSD,South Sudan,2023,160,3.220970,3.505,2.500,4.487,1.396,5.0,...,4.999697,1.037000,1.000,5.000,4.0,1.0,1.000,2.347082,3.835324,3.080335
2733,SYR,Syria,2023,161,3.294064,3.721,1.053,4.971,1.475,5.0,...,1.000000,1.256000,1.000,5.000,5.0,1.0,1.000,2.115765,3.841971,3.459541
2734,YEM,Yemen,2023,162,3.349941,3.174,5.000,3.192,1.277,5.0,...,2.995442,1.000000,1.000,5.000,4.0,1.3,1.000,2.234993,3.900088,3.485752


In [41]:
find_different_countries(df, safety)

159
Database differences: ['Belize', 'Nauru', 'Reunion', 'San Marino', 'Tonga', 'Micronesia', 'French Guiana', 'Cook Islands', 'Northern Mariana Islands', 'French Polynesia', 'Cayman Islands', 'Sao Tome And Principe', 'Sint Eustatius', 'United States Virgin Islands', 'Maldives', 'Turkey', 'Tuvalu', 'Niue', 'Czech Republic', 'Congo, Democratic Republic Of The', 'Brunei Darussalam', 'Sint Maarten (Dutch Part)', 'Congo', 'New Caledonia', 'Puerto Rico', 'Kiribati', 'Fiji', 'Bonaire', 'Comoros', 'Antigua And Barbuda', 'Bahamas', 'Dominica', 'British Virgin Islands', 'Guadeloupe', 'State Of Palestine', 'Guam', 'Cote DIvoire', 'Anguilla', 'Saba', 'Saint Lucia', 'Saint Kitts And Nevis', 'Gambia', 'Turks And Caicos Islands', 'Saint Vincent And The Grenadines', 'Bermuda', 'American Samoa', 'Seychelles', 'Palau', 'Solomon Islands', 'Montserrat', 'Kyrgyzstan', 'Curaçao', 'Macao, China', 'Grenada', 'Hong Kong, China', 'Andorra', 'Martinique', 'Taiwan Province Of China', 'Vanuatu', 'Syrian Arab Repu

In [42]:
# renaming countries in safety segmentation
country_mapping = {
    'The Gambia': 'Gambia',
    'Syria' : 'Syrian Arab Republic',
    'Czechia': 'Czech Republic',
    'Democratic Republic Of The Congo': 'Congo, Democratic Republic Of The',
    'Republic Of The Congo': 'Congo',
    'Taiwan': 'Taiwan Province Of China',
    "Cote D' Ivoire": 'Cote Divoire',
    'Turkiye': 'Turkey',
    'Kyrgyz Republic': 'Kyrgyzstan',
    'Palestine': 'State of Palestine'
}

safety['country'] = safety['country'].replace(country_mapping)

In [43]:
df_hist = pd.merge(df_hist, safety, on=['country', 'year'], how='left')
df_hist

,country,year,inbounds_tourists,inbounds_total,region,avg_length_stay,beds_number,establishments_number,rooms_number,beds_occupancy_rate,rooms_occupancy_rate,seniors,senior_%,peace_index,terrorism_index
0,Afghanistan,1995,NaN,NaN,South Asia,NaN,NaN,NaN,NaN,NaN,NaN,373005.5,2271.985760,NaN,NaN
1,Afghanistan,1996,NaN,NaN,South Asia,NaN,NaN,NaN,NaN,NaN,NaN,389379.5,2276.379536,NaN,NaN
2,Afghanistan,1997,NaN,NaN,South Asia,NaN,NaN,NaN,NaN,NaN,NaN,405276.5,2278.452291,NaN,NaN
3,Afghanistan,1998,NaN,NaN,South Asia,NaN,NaN,NaN,NaN,NaN,NaN,421195.0,2277.764389,NaN,NaN
4,Afghanistan,1999,NaN,NaN,South Asia,NaN,NaN,NaN,NaN,NaN,NaN,438599.0,2277.108780,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6239,Zimbabwe,2018,NaN,2580000.0,Africa,3.0,12772.0,96.0,6483.0,39.0,53.0,486199.5,3232.137585,2.200260,0.0
6240,Zimbabwe,2019,NaN,2294000.0,Africa,3.0,13158.0,97.0,6676.0,32.0,44.0,503951.5,3284.178333,2.309794,0.0
6241,Zimbabwe,2020,NaN,639000.0,Africa,3.0,14551.0,95.0,7217.0,15.0,19.0,519181.5,3315.377909,2.321812,0.0
6242,Zimbabwe,2021,NaN,381000.0,Africa,3.0,14551.0,95.0,7217.0,18.0,27.0,528314.0,3305.284104,2.305928,0.0


# Yearly variables - Weight calculation

In [44]:
df_ranking = df_hist.query('year == 2022')

In [45]:
# visa
visa.rename(columns = {'Country': 'country', 'Tourism Openness Index': 'visa_index'}, inplace = True)

In [46]:
find_different_countries(df_ranking, visa)

200
Database differences: ['Laos', 'Northern Mariana Islands', 'Eswatini', 'Sao Tome And Principe', 'United States Virgin Islands', 'Guinea', 'Turkey', 'Sint Maarten (Dutch Part)', 'Brunei Darussalam', 'Congo, Democratic Republic Of The', 'Slovakia', 'North Macedonia', 'Comoros', 'State Of Palestine', 'Guam', 'Cote DIvoire', 'South Korea', 'Vietnam', 'American Samoa', 'North Korea', 'Russia', 'South Sudan']
New variable differences: ['Congo, Democratic Republic Of', 'Korea, Republic Of', 'Guinea, Republic Of', 'Lao People’S Democratic Republic', 'Sint Maarten', 'Macedonia', 'Sao Tome E Principe', 'Brunei', 'Russian Federation', 'Slovak Republic', 'Comoros Islands', 'Viet Nam', 'Türkiye', 'Swaziland', "Korea, Democratic People'S Republic Of", "Côte D'Ivoire"]


In [47]:
country_mapping = {

    'Russian Federation': 'Russia',
    'Guinea, Republic Of': 'Guinea',
    'Lao People’S Democratic Republic': 'Laos',
    'Türkiye': 'Turkey',
    'Viet Nam': 'Vietnam',
    "Côte D'Ivoire": 'Cote Divoire',
    'Congo, Democratic Republic Of': 'Congo, Democratic Republic Of The',
    'Slovak Republic': 'Slovakia',
    'Korea, Republic Of': 'South Korea',
    "Korea, Democratic People'S Republic Of": 'North Korea',
    'Macedonia': 'North Macedonia',
    'Swaziland': 'Eswatini',
    'Comoros Islands': 'Comoros',
    'Brunei': 'Brunei Darussalam',
    'Sao Tome E Principe': 'Sao Tome And Principe',
    'Sint Marteen': 'Sint Marteen (Dutch Part)',
    'Spain': 'Spain'
}

visa['country'] = visa['country'].replace(country_mapping)

In [48]:
df_ranking = pd.merge(df_ranking, visa, on=['country'], how='left')

Environment

In [49]:
# Environment
find_different_countries(df_ranking, env)

175
Database differences: ['Nauru', 'Reunion', 'San Marino', 'French Guiana', 'Cook Islands', 'Northern Mariana Islands', 'French Polynesia', 'Cayman Islands', 'Sint Eustatius', 'United States Virgin Islands', 'Tuvalu', 'Niue', 'Sint Maarten (Dutch Part)', 'Congo, Democratic Republic Of The', 'Congo', 'New Caledonia', 'Puerto Rico', 'Bonaire', 'Libya', 'British Virgin Islands', 'Guadeloupe', 'State Of Palestine', 'Guam', 'Cote DIvoire', 'Anguilla', 'Saba', 'Yemen', 'Saint Kitts And Nevis', 'South Sudan', 'Turks And Caicos Islands', 'Vietnam', 'Bermuda', 'Somalia', 'American Samoa', 'Liechtenstein', 'Palau', 'North Korea', 'Montserrat', 'Curaçao', 'Macao, China', 'Hong Kong, China', 'Andorra', 'Martinique', 'Taiwan Province Of China', 'Syrian Arab Republic', 'Monaco', 'Aruba']
New variable differences: ["Cote D'Ivoire", 'Taiwan', 'Viet Nam', 'Republic Of Congo', 'Dem. Rep. Congo']


In [50]:
country_mapping = {
    'Taiwan': 'Taiwan Province Of China',
    'Republic Of Congo': 'Congo',
    'Dem. Rep. Congo': 'Congo, Democratic Republic Of The',
    'Viet Nam': 'Vietnam',
    "Cote D'Ivoire": 'Cote Divoire',
}

env['country'] = env['country'].replace(country_mapping)

In [51]:
df_ranking = pd.merge(df_ranking, env, on=['country'], how='left')

World Heritage

In [52]:
# World Heritage
find_different_countries(df_ranking, wh)

151
Database differences: ['Nauru', 'Reunion', 'Sierra Leone', 'Burundi', 'Timor-Leste', 'Tonga', 'Micronesia', 'Bhutan', 'Cook Islands', 'French Guiana', 'Northern Mariana Islands', 'Eswatini', 'Tanzania', 'Cayman Islands', 'Sint Eustatius', 'Laos', 'United Kingdom', 'Maldives', 'Tuvalu', 'Niue', 'Turkey', 'Congo, Democratic Republic Of The', 'Czech Republic', 'Brunei Darussalam', 'Sint Maarten (Dutch Part)', 'Liberia', 'New Caledonia', 'Moldova', 'Guyana', 'Puerto Rico', 'Bonaire', 'Comoros', 'Trinidad And Tobago', 'Bahamas', 'British Virgin Islands', 'Guadeloupe', 'Equatorial Guinea', 'Guam', 'Cote DIvoire', 'Iran', 'Anguilla', 'Saba', 'French Polynesia', 'South Korea', 'Djibouti', 'Guinea-Bissau', 'Kuwait', 'South Sudan', 'Turks And Caicos Islands', 'Saint Vincent And The Grenadines', 'Vietnam', 'Bermuda', 'Somalia', 'American Samoa', 'Liechtenstein', 'Bolivia', 'North Korea', 'Montserrat', 'Curaçao', 'United States Virgin Islands', 'Venezuela', 'Macao, China', 'Grenada', 'Hong Kon

In [53]:
country_mapping = {

    'United Kingdom Of Great Britain And Northern Ireland': 'United Kingdom',
    'Russian Federation': 'Russia', 
    'Venezuela (Bolivarian Republic Of)': 'Venezuela',
    "Lao People'S Democratic Republic": 'Laos',
    'Republic Of Korea': 'South Korea',
    "Democratic People'S Republic Of Korea": 'North Korea',
    'Iran (Islamic Republic Of)': 'Iran',
    'Democratic Republic Of The Congo': 'Congo, Democratic Republic Of The',
    'Türkiye': 'Turkey',
    'Viet Nam': 'Vietnam',
    'Republic Of Moldova': 'Moldova',
    'United Republic Of Tanzania': 'Tanzania', 
    'Bolivia (Plurinational State Of)': 'Bolivia',
    "Côte D'Ivoire": 'Cote Divoire',
    'Czechia': 'Czech Republic',
    'Micronesia (Federated States Of)': 'Micronesia'
}

wh['country'] = wh['country'].replace(country_mapping)

In [54]:
df_ranking = pd.merge(df_ranking, wh[['country', 'wh_sites']], on=['country'], how='left')

In [55]:
amusement = amusement.rename(columns= {'Country': 'country', 'Amusement parks': 'amusement_parks'})
cuisine = cuisine.drop('Unnamed: 0', axis = 1).rename(columns = {'Country': 'country', 'Ranking': 'cuisine_ranking'})
cuisine['country'] = cuisine['country'].replace({'-': ' '}, regex = True)
michelin.drop('Unnamed: 0', axis = 1, inplace = True)

Amusement

In [56]:
find_different_countries(df_ranking, amusement)

80
Database differences: ['Marshall Islands', 'Reunion', 'San Marino', 'Micronesia', 'Tunisia', 'Tuvalu', 'Niue', 'Kiribati', 'Peru', 'Bahamas', 'Dominica', 'Ethiopia', 'State Of Palestine', 'Burkina Faso', 'Albania', 'Georgia', 'Yemen', 'Kenya', 'Somalia', 'American Samoa', 'Nepal', 'Zambia', 'Palau', 'Montserrat', 'Panama', 'Andorra', 'Vanuatu', 'Monaco', 'Senegal', 'Iraq', 'Belize', 'Nauru', 'Lesotho', 'Sierra Leone', 'Bhutan', 'Cook Islands', 'French Guiana', 'Northern Mariana Islands', 'Maldives', 'Zimbabwe', 'Congo, Democratic Republic Of The', 'North Macedonia', 'Congo', 'Fiji', 'Bonaire', 'Comoros', 'Namibia', 'British Virgin Islands', 'Afghanistan', 'Rwanda', 'Saba', 'Nicaragua', 'Haiti', 'Jordan', 'Eritrea', 'Saint Lucia', 'Uruguay', 'Turks And Caicos Islands', 'Bermuda', 'Seychelles', 'Bolivia', 'Kyrgyzstan', 'Bulgaria', 'Angola', 'Grenada', 'Hong Kong, China', 'South Sudan', 'Oman', 'Mauritania', 'Lebanon', 'United States Of America', 'Benin', 'Burundi', 'Timor-Leste', 'Ton

In [57]:
country_mapping = {

    'United States': 'United States Of America',
    'Hong Kong': 'Hong Kong, China',
    'Taiwan': 'Taiwan Province Of China',
    'Brunei': 'Brunei Darussalam'
}

amusement['country'] = amusement['country'].replace(country_mapping)

In [58]:
df_ranking = pd.merge(df_ranking, amusement, on=['country'], how='left')

Cuisine

In [59]:
find_different_countries(df_ranking, cuisine)

88
Database differences: ['Marshall Islands', 'Reunion', 'San Marino', 'Micronesia', 'United Kingdom', 'Tuvalu', 'Niue', 'Kiribati', 'Luxembourg', 'Bahamas', 'Dominica', 'Sudan', 'State Of Palestine', 'Burkina Faso', 'Yemen', 'Kuwait', 'Kenya', 'Somalia', 'American Samoa', 'Nepal', 'Zambia', 'Palau', 'North Korea', 'Montserrat', 'Panama', 'Andorra', 'Vanuatu', 'Monaco', 'Senegal', 'Belize', 'Nauru', 'Lesotho', 'Sierra Leone', 'Bhutan', 'Cook Islands', 'French Guiana', 'Northern Mariana Islands', 'Maldives', 'Turkey', 'Zimbabwe', 'Congo, Democratic Republic Of The', 'Bahrain', 'Congo', 'Fiji', 'Bonaire', 'Comoros', 'Namibia', 'British Virgin Islands', 'Afghanistan', 'Rwanda', 'Saba', 'Nicaragua', 'Eritrea', 'Saint Lucia', 'Turks And Caicos Islands', 'Bermuda', 'Botswana', 'Seychelles', 'Turkmenistan', 'Angola', 'Grenada', 'Hong Kong, China', 'South Sudan', 'Oman', 'Mauritania', 'United States Of America', 'Benin', 'Malta', 'Burundi', 'Costa Rica', 'Timor-Leste', 'Tonga', 'Eswatini', 'Ta

In [60]:
country_mapping = {

    'Usa': 'United States Of America',
    'England': 'United Kingdom',
    'Taiwan': 'Taiwan Province Of China',
    'Turkiye': 'Turkey',
    'Korea': 'South Korea',
    'Syria': 'Syrian Arab Republic',
    # 'Northern Ireland': 'Ireland',
    'Palestine': 'State Of Palestine',
    'The Bahamas': 'Bahamas'
}

cuisine['country'] = cuisine['country'].replace(country_mapping)

In [61]:
df_ranking = pd.merge(df_ranking, cuisine, on=['country'], how='left')

Michelin

In [62]:
find_different_countries(df_ranking, michelin)

40
Database differences: ['Marshall Islands', 'Mexico', 'Cyprus', 'Pakistan', 'Reunion', 'Indonesia', 'Micronesia', 'Tunisia', 'Niue', 'Tuvalu', 'Slovakia', 'Armenia', 'India', 'Saudi Arabia', 'Kiribati', 'Peru', 'Bahamas', 'Dominica', 'Ethiopia', 'Sudan', 'Burkina Faso', 'State Of Palestine', 'Albania', 'Iran', 'Georgia', 'Yemen', 'Kuwait', 'Bangladesh', 'Chile', 'Kenya', 'Somalia', 'American Samoa', 'Nepal', 'Zambia', 'Palau', 'North Korea', 'Montserrat', 'Panama', 'Venezuela', 'Vanuatu', 'Azerbaijan', 'Senegal', 'Iraq', 'Qatar', 'Belize', 'Nauru', 'Lesotho', 'Sierra Leone', 'Bhutan', 'Cook Islands', 'French Guiana', 'Northern Mariana Islands', 'Maldives', 'Zimbabwe', 'Congo, Democratic Republic Of The', 'Bahrain', 'Congo', 'North Macedonia', 'Fiji', 'Bonaire', 'Comoros', 'Namibia', 'Colombia', 'British Virgin Islands', 'Afghanistan', 'Belarus', 'Rwanda', 'Saba', 'Nicaragua', 'Haiti', 'Jordan', 'Eritrea', 'Cuba', 'Saint Lucia', 'Uruguay', 'Turks And Caicos Islands', 'Bermuda', 'Botsw

In [63]:
country_mapping = {

    'United States': 'United States Of America',
    'Hong Kong': 'Hong Kong, China',
    'Taiwan': 'Taiwan Province Of China',
    'Macau': 'Macao, China'
}

michelin['country'] = michelin['country'].replace(country_mapping)

In [64]:
df_ranking = pd.merge(df_ranking, michelin[['country', 'michelin_stars']], on=['country'], how='left')

### Correlations

In [65]:
df_ranking = df_ranking.drop_duplicates()

In [66]:
prueba = df_ranking.drop(['country', 'year', 'region'], axis = 1)
prueba.corr()

,inbounds_tourists,inbounds_total,avg_length_stay,beds_number,establishments_number,rooms_number,beds_occupancy_rate,rooms_occupancy_rate,seniors,senior_%,...,hlt_score,air_score,usd_score,rec_score,ocp_score,cch_score,wh_sites,amusement_parks,cuisine_ranking,michelin_stars
inbounds_tourists,1.000000,0.912729,-0.212103,0.896999,0.595337,0.891351,0.209898,0.219805,0.614210,0.314531,...,0.405601,0.359307,0.432781,0.250209,-0.318487,0.001279,0.827056,0.453588,-0.533630,0.655745
inbounds_total,0.912729,1.000000,-0.223432,0.488618,0.300871,0.532893,0.288362,0.139198,0.170912,0.337503,...,0.291946,0.204698,0.387198,0.228082,-0.217540,0.046072,0.682835,0.355618,-0.488787,0.317325
avg_length_stay,-0.212103,-0.223432,1.000000,-0.166035,-0.154449,-0.207971,0.615400,0.388798,-0.212457,-0.308448,...,-0.325064,-0.299381,-0.313265,-0.313968,0.307497,-0.086476,-0.236080,-0.192783,0.420095,-0.120550
beds_number,0.896999,0.488618,-0.166035,1.000000,0.859290,0.988335,0.106360,0.180398,0.481774,0.289580,...,0.293688,0.257300,0.339553,0.140363,-0.472034,-0.162818,0.775933,0.612045,-0.569233,0.709977
establishments_number,0.595337,0.300871,-0.154449,0.859290,1.000000,0.865122,0.098890,0.075304,0.238039,0.192882,...,0.183725,0.161865,0.180881,0.036254,-0.401361,-0.188489,0.570948,0.470743,-0.561736,0.726343
rooms_number,0.891351,0.532893,-0.207971,0.988335,0.865122,1.000000,0.139914,0.108676,0.529747,0.278255,...,0.267760,0.224270,0.320755,0.167948,-0.496429,-0.176497,0.805631,0.602427,-0.578358,0.712831
beds_occupancy_rate,0.209898,0.288362,0.615400,0.106360,0.098890,0.139914,1.000000,0.761375,0.073835,0.036579,...,0.145054,0.141518,0.137833,0.083380,0.132736,-0.134173,0.108620,-0.034383,-0.282621,0.044922
rooms_occupancy_rate,0.219805,0.139198,0.388798,0.180398,0.075304,0.108676,0.761375,1.000000,-0.045474,0.322222,...,0.495162,0.475711,0.466704,0.297957,0.231730,0.082281,0.172691,0.146791,-0.060379,0.120025
seniors,0.614210,0.170912,-0.212457,0.481774,0.238039,0.529747,0.073835,-0.045474,1.000000,0.108222,...,0.015224,-0.024928,0.070074,0.038084,-0.338868,-0.128227,0.592939,0.306247,-0.341632,0.236877
senior_%,0.314531,0.337503,-0.308448,0.289580,0.192882,0.278255,0.036579,0.322222,0.108222,1.000000,...,0.794316,0.711261,0.788612,0.358960,-0.005430,0.448674,0.435459,0.164914,-0.425892,0.315196


Correlación amusement parks quitando el outlier de US = 0.293470
Correlación amusement parks con US = 0.31

In [67]:
correlations = prueba.corr()[['inbounds_tourists']]
correlations

,inbounds_tourists
inbounds_tourists,1.000000
inbounds_total,0.912729
avg_length_stay,-0.212103
beds_number,0.896999
establishments_number,0.595337
rooms_number,0.891351
beds_occupancy_rate,0.209898
rooms_occupancy_rate,0.219805
seniors,0.614210
senior_%,0.314531


# 2024 Ranking Creation

In [68]:
vars = ['senior_%', 'terrorism_index', 'visa_index', 'epi_score', 'usd_score', 'wh_sites', 'amusement_parks', 'cuisine_ranking']
weights1 = correlations.loc[vars].reset_index().rename(columns = {'index': 'variable', 'inbounds_tourists': 'weight'})
# weights1.to_clipboard()

# turn all weights1 into absolute value so the user has it easier to interact with them
weights1['weight'] = weights1['weight'].abs().round(2)
weights1['use'] = True

In [69]:
other_vars = ['peace_index', 'michelin_stars', 'eco_score', 'bdh_score', 'hlt_score', 'air_score', 'rec_score', 'ocp_score', 'cch_score']

weights2 = correlations.loc[other_vars].reset_index().rename(columns = {'index': 'variable', 'inbounds_tourists': 'weight'})
# weights2.to_clipboard()

# turn all weights2 into absolute value so the user has it easier to interact with them
weights2['weight'] = weights2['weight'].abs().round(2)
weights2['use'] = False


In [70]:
weights = pd.concat([weights1, weights2])
weights

,variable,weight,use
0,senior_%,0.31,True
1,terrorism_index,0.18,True
2,visa_index,0.28,True
3,epi_score,0.25,True
4,usd_score,0.43,True
5,wh_sites,0.83,True
6,amusement_parks,0.45,True
7,cuisine_ranking,0.53,True
0,peace_index,0.06,False
1,michelin_stars,0.66,False


In [103]:
weights.variable = ['Senior Population', 'Terrorism', 'Visa Openness', 'Environmental Score', 'Sanitation', 'World Heritage Sites', 'Amusement Parks', 'Cuisine', 'Safety', 'Michelin Stars', 
                   'Ecosystem Vitality', 'Biodiversity & Habitat', 'Environmental Health', 'Air Quality', 'Recycling', 'Ocean Plastics', 'Climate Change Policy']

In [104]:
weights

,variable,weight,use
0,Senior Population,0.31,True
1,Terrorism,0.18,True
2,Visa Openness,0.28,True
3,Environmental Score,0.25,True
4,Sanitation,0.43,True
5,World Heritage Sites,0.83,True
6,Amusement Parks,0.45,True
7,Cuisine,0.53,True
0,Safety,0.06,False
1,Michelin Stars,0.66,False


In [15]:
weights.to_csv('data/processed/default_weights.csv')

In [3]:
weights = pd.read_csv('data/processed/default_weights.csv').drop('Unnamed: 0', axis = 1)

In [85]:
# creating the dataframe with all variables except the safety ones
final_vars = ['country', 'visa_index', 'epi_score', 'usd_score', 'wh_sites', 'amusement_parks', 'cuisine_ranking',
              'michelin_stars', 'eco_score', 'bdh_score', 'hlt_score', 'air_score', 'rec_score', 'ocp_score', 'cch_score', 'senior_%'
              ]
df_24 = df_ranking[final_vars].drop_duplicates()
# adding the safety variables from 2023
df_24 = pd.merge(df_24, safety.query('year == 2023')[['country', 'terrorism_index', 'peace_index']], on = 'country')

In [72]:
df_24

,country,visa_index,epi_score,usd_score,wh_sites,amusement_parks,cuisine_ranking,michelin_stars,eco_score,bdh_score,hlt_score,air_score,rec_score,ocp_score,cch_score,senior_%,terrorism_index,peace_index
0,Afghanistan,0.00,43.6,28.4,2.0,NaN,NaN,NaN,36.9,30.7,16.0,15.5,11.1,NaN,65.6,2381.698566,7.825,3.448482
1,Albania,71.64,47.1,59.6,4.0,NaN,91.0,NaN,45.5,63.9,40.0,37.5,20.5,32.1,52.5,16330.087095,0.000,1.744803
2,Algeria,1.47,29.6,59.6,7.0,2.0,40.0,NaN,31.6,22.7,42.0,39.4,11.0,10.5,20.9,6296.737773,2.197,2.093692
3,Angola,51.00,30.5,14.1,1.0,NaN,NaN,NaN,28.6,30.1,20.5,23.1,19.0,19.5,37.7,2579.874502,2.254,2.020407
4,Argentina,28.32,41.1,72.6,12.0,2.0,14.0,NaN,38.9,42.4,56.3,52.0,9.6,17.7,35.5,11410.854594,1.274,1.836749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Venezuela,25.22,46.4,51.6,3.0,8.0,70.0,NaN,52.0,71.5,42.9,46.7,21.3,14.0,42.1,8534.008505,1.174,2.692722
163,Vietnam,42.46,20.1,55.7,8.0,NaN,22.0,4.0,22.1,27.9,35.1,26.5,46.0,12.3,10.1,8880.196813,0.000,1.745043
164,Yemen,2.46,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2643.066159,4.951,3.349941
165,Zambia,72.61,38.4,13.9,1.0,NaN,NaN,NaN,58.2,91.0,21.2,23.6,12.6,NaN,25.6,1732.660523,0.000,1.898428


Hay que scale las variables para que todas estén entre 0 y 1, sino los pesos no valen

In [86]:
from sklearn.preprocessing import MinMaxScaler
# scaling all values to be between 0 and 1
scaler_usado = MinMaxScaler()
scaler = scaler_usado.fit(df_24[vars])
df_24[vars] = scaler.transform(df_24[vars])

# reversing variables with a negative correlation so the absolute numbers work
df_24['cuisine_ranking'] = 1 - df_24['cuisine_ranking']
df_24['terrorism_index'] = 1 - df_24['terrorism_index']

In [88]:
df_24.columns = ['Country', 'Visa Openness', 'Environmental Score', 'Sanitation', 'World Heritage Sites', 'Amusement Parks', 'Cuisine', 'Michelin Stars', 
                 'Ecosystem Vitality', 'Biodiversity & Habitat', 'Environmental Health', 'Air Quality', 'Recycling', 'Ocean Plastics', 'Climate Change Policy',
                 'Senior Population', 'Terrorism', 'Safety']

In [89]:
df_24.to_csv('data/processed/final_df.csv', index = False)

In [99]:
df_24

,Country,Visa Openness,Environmental Score,Sanitation,World Heritage Sites,Amusement Parks,Cuisine,Michelin Stars,Ecosystem Vitality,Biodiversity & Habitat,Environmental Health,Air Quality,Recycling,Ocean Plastics,Climate Change Policy,Senior Population,Terrorism,Safety
0,Afghanistan,0.000000,0.418644,0.274569,0.017241,NaN,NaN,NaN,36.9,30.7,16.0,15.5,11.1,NaN,65.6,0.032832,0.087038,3.448482
1,Albania,0.747262,0.477966,0.590679,0.051724,NaN,0.090909,NaN,45.5,63.9,40.0,37.5,20.5,32.1,52.5,0.555575,1.000000,1.744803
2,Algeria,0.015333,0.181356,0.590679,0.103448,0.002725,0.606061,NaN,31.6,22.7,42.0,39.4,11.0,10.5,20.9,0.179556,0.743671,2.093692
3,Angola,0.531970,0.196610,0.129686,0.000000,NaN,NaN,NaN,28.6,30.1,20.5,23.1,19.0,19.5,37.7,0.040260,0.737020,2.020407
4,Argentina,0.295400,0.376271,0.722391,0.189655,0.002725,0.868687,NaN,38.9,42.4,56.3,52.0,9.6,17.7,35.5,0.371218,0.851359,1.836749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Venezuela,0.263065,0.466102,0.509625,0.034483,0.019074,0.303030,NaN,52.0,71.5,42.9,46.7,21.3,14.0,42.1,0.263402,0.863026,2.692722
163,Vietnam,0.442891,0.020339,0.551165,0.120690,NaN,0.787879,4.0,22.1,27.9,35.1,26.5,46.0,12.3,10.1,0.276376,1.000000,1.745043
164,Yemen,0.025660,NaN,NaN,0.068966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042628,0.422354,3.349941
165,Zambia,0.757380,0.330508,0.127660,0.000000,NaN,NaN,NaN,58.2,91.0,21.2,23.6,12.6,NaN,25.6,0.008509,1.000000,1.898428


In [105]:
salida = df_24.melt('Country')
salida = salida.merge(weights, on = 'variable')
# .rename(columns = {'inbounds_tourists': 'coef'})

In [106]:
salida['kpi'] = salida.value * salida.weight
ranking = salida.groupby('Country').sum()[['kpi']].reset_index()
ranking = ranking.sort_values(by = 'kpi', ascending= False)
ranking['rank'] = range(1, len(ranking) + 1)

In [107]:
ranking['rank2'] = ranking['kpi'].rank(method='min', ascending=False).astype(int)

In [108]:
ranking

,Country,kpi,rank,rank2
51,France,533.184195,1,1
74,Japan,378.850519,2,2
72,Italy,359.137994,3,3
55,Germany,339.533691,4,4
138,Spain,276.239702,5,5
...,...,...,...,...
111,North Korea,0.487985,163,163
85,Libya,0.460530,164,164
134,Somalia,0.414682,165,165
137,South Sudan,0.389253,166,166


In [436]:
ranking.sort_values(by = 'kpi', ascending= False).to_clipboard()

In [77]:
def obtener_ranking(df_24, weights):
    salida = df_24.melt('country')
    salida = salida.merge(weights, on = 'variable')
    salida['kpi'] = salida.value * salida.weight
    ranking = salida.groupby('country').sum()[['kpi']].reset_index()
    ranking = ranking.sort_values(by = 'kpi', ascending= False)
    ranking['rank'] = range(1, len(ranking) + 1)
    return ranking

In [79]:
df_24

,country,senior_%,visa_index,epi_score,usd_score,wh_sites,amusement_parks,michelin_stars,cuisine_ranking,terrorism_index
0,Afghanistan,0.032832,1.000000,0.418644,0.274569,0.017241,NaN,NaN,NaN,0.912962
1,Albania,0.555575,0.252738,0.477966,0.590679,0.051724,NaN,NaN,0.090909,0.000000
2,Algeria,0.179556,0.984667,0.181356,0.590679,0.103448,0.002725,NaN,0.606061,0.256329
3,Angola,0.040260,0.468030,0.196610,0.129686,0.000000,NaN,NaN,NaN,0.262980
4,Argentina,0.371218,0.704600,0.376271,0.722391,0.189655,0.002725,NaN,0.868687,0.148641
...,...,...,...,...,...,...,...,...,...,...
162,Venezuela,0.263402,0.736935,0.466102,0.509625,0.034483,0.019074,NaN,0.303030,0.136974
163,Vietnam,0.276376,0.557109,0.020339,0.551165,0.120690,NaN,0.0048,0.787879,0.000000
164,Yemen,0.042628,0.974340,NaN,NaN,0.068966,NaN,NaN,NaN,0.577646
165,Zambia,0.008509,0.242620,0.330508,0.127660,0.000000,NaN,NaN,NaN,0.000000


In [81]:
weights['weight'].round(2)

0    0.42
1    0.22
2    0.39
3    0.28
4    0.52
5    0.82
6    0.31
7    0.63
8    0.61
Name: weight, dtype: float64

In [78]:
obtener_ranking(df_24, weights)['country'].head(5)

51      France
72       Italy
55     Germany
74       Japan
138      Spain
Name: country, dtype: object

In [74]:
prueba = obtener_ranking(weights, df_24)

In [76]:
prueba

,country,kpi,rank
51,France,3.343150,1
72,Italy,3.201153,2
55,Germany,2.937715,3
74,Japan,2.840245,4
138,Spain,2.826284,5
...,...,...,...
22,Burundi,0.254225,163
95,Mauritania,0.251558,164
60,Guinea-Bissau,0.248321,165
89,Madagascar,0.227313,166


In [75]:
prueba['country'].head(5)

51      France
72       Italy
55     Germany
74       Japan
138      Spain
Name: country, dtype: object

In [66]:
obtener_ranking(weights, df_24)

,country,kpi,rank
51,France,3.343150,1
72,Italy,3.201153,2
55,Germany,2.937715,3
74,Japan,2.840245,4
138,Spain,2.826284,5
...,...,...,...
22,Burundi,0.254225,163
95,Mauritania,0.251558,164
60,Guinea-Bissau,0.248321,165
89,Madagascar,0.227313,166


In [437]:
ranking

,country,kpi
0,Afghanistan,0.874656
1,Albania,0.867093
2,Algeria,1.325916
3,Angola,0.377780
4,Argentina,1.627902
...,...,...
163,Venezuela,1.038766
164,Vietnam,1.207145
165,Yemen,0.579858
166,Zambia,0.255020


In [397]:
import plotly.express as px

In [442]:
w_graph = weights.sort_values(by = 'weight', ascending = False)

fig = px.bar(x = w_graph['variable'], y = w_graph['weight'], barmode = 'stack', color_discrete_sequence=['blue'])
fig.update_layout(yaxis_title=f'Weight', xaxis_title = 'Variables', width = 800, height = 600)

fig.update_traces(marker=dict(line=dict(width=2))) 

fig.show()
# return fig

In [364]:
salida

,country,variable,value,coef,kpi
0,Afghanistan,senior_%,0.028911,0.421416,0.012183
1,Albania,senior_%,0.489214,0.421416,0.206163
2,Algeria,senior_%,0.158109,0.421416,0.066630
3,American Samoa,senior_%,0.188474,0.421416,0.079426
4,Andorra,senior_%,0.421743,0.421416,0.177729
...,...,...,...,...,...
1771,Venezuela,michelin_stars,NaN,0.629084,NaN
1772,Vietnam,michelin_stars,0.004800,0.629084,0.003020
1773,Yemen,michelin_stars,NaN,0.629084,NaN
1774,Zambia,michelin_stars,NaN,0.629084,NaN


In [ ]:
pd.read_csv('data/processed/default_weights.csv').drop('Unnamed: 0', axis = 1)

,variable,weight,use
0,senior_%,0.32,True
1,terrorism_index,0.18,True
2,visa_index,0.29,True
3,epi_score,0.25,True
4,usd_score,0.44,True
5,wh_sites,0.84,True
6,amusement_parks,0.46,True
7,cuisine_ranking,0.54,True
8,peace_index,0.06,False
9,michelin_stars,0.66,False


In [ ]:
pd.read_csv('final_df.csv')

,country,senior_%,visa_index,epi_score,usd_score,wh_sites,amusement_parks,michelin_stars,cuisine_ranking,terrorism_index
0,Afghanistan,0.032832,1.000000,0.418644,0.274569,0.017241,NaN,NaN,NaN,0.912962
1,Albania,0.555575,0.252738,0.477966,0.590679,0.051724,NaN,NaN,0.090909,0.000000
2,Algeria,0.179556,0.984667,0.181356,0.590679,0.103448,0.002725,NaN,0.606061,0.256329
3,Angola,0.040260,0.468030,0.196610,0.129686,0.000000,NaN,NaN,NaN,0.262980
4,Argentina,0.371218,0.704600,0.376271,0.722391,0.189655,0.002725,NaN,0.868687,0.148641
...,...,...,...,...,...,...,...,...,...,...
162,Venezuela,0.263402,0.736935,0.466102,0.509625,0.034483,0.019074,NaN,0.303030,0.136974
163,Vietnam,0.276376,0.557109,0.020339,0.551165,0.120690,NaN,0.0048,0.787879,0.000000
164,Yemen,0.042628,0.974340,NaN,NaN,0.068966,NaN,NaN,NaN,0.577646
165,Zambia,0.008509,0.242620,0.330508,0.127660,0.000000,NaN,NaN,NaN,0.000000
